In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import math
import heapq as heap
import time

In [62]:
df = pd.read_csv('Dataset.csv')

In [63]:
df.drop_duplicates(subset=['SourceAirport', 'DestinationAirport', 'SourceAirport_City', 'SourceAirport_Country',
                               'SourceAirport_Latitude', 'SourceAirport_Longitude', 'SourceAirport_Altitude',
                               'DestinationAirport_City', 'DestinationAirport_Country', 'DestinationAirport_Latitude',
                               'DestinationAirport_Longitude', 'DestinationAirport_Altitude', 'Distance', 'FlyTime', 'Price'], ignore_index=True, inplace=True)


In [64]:
# Create a new column 'SourceAirport_Location' by combining the latitude, longitude, and altitude into a tuple
df['SourceAirport_Location'] = df.apply(lambda row: (row['SourceAirport_Latitude'], row['SourceAirport_Longitude'], row['SourceAirport_Altitude']), axis=1)

# Drop the individual columns since you now have a single combined column
df.drop(['SourceAirport_Latitude', 'SourceAirport_Longitude', 'SourceAirport_Altitude'], axis=1, inplace=True)


In [65]:
# Create a new column 'DestinationAirport_Location' by combining the latitude, longitude, and altitude into a tuple
df['DestinationAirport_Location'] = df.apply(lambda row: (row['DestinationAirport_Latitude'], row['DestinationAirport_Longitude'], row['DestinationAirport_Altitude']), axis=1)

# Drop the individual columns since you now have a single combined column
df.drop(['DestinationAirport_Latitude', 'DestinationAirport_Longitude', 'DestinationAirport_Altitude'], axis=1, inplace=True)


In [66]:
df

,Airline,SourceAirport,DestinationAirport,SourceAirport_City,SourceAirport_Country,DestinationAirport_City,DestinationAirport_Country,Distance,FlyTime,Price,SourceAirport_Location,DestinationAirport_Location
0,Aerocondor,Sochi International Airport,Kazan International Airport,Sochi,Russia,Kazan,Russia,1506.825604,2.371833,723.412802,"(43.449902, 39.9566, 89)","(55.606201171875, 49.278701782227, 411)"
1,Aerocondor,Astrakhan Airport,Kazan International Airport,Astrakhan,Russia,Kazan,Russia,1040.438320,1.792470,490.219160,"(46.2832984924, 48.0063018799, -65)","(55.606201171875, 49.278701782227, 411)"
2,Aerocondor,Chelyabinsk Balandino Airport,Kazan International Airport,Chelyabinsk,Russia,Kazan,Russia,770.508500,1.457153,355.254250,"(55.305801, 61.5033, 769)","(55.606201171875, 49.278701782227, 411)"
3,Aerocondor,Domodedovo International Airport,Kazan International Airport,Moscow,Russia,Kazan,Russia,715.649350,1.389005,327.824675,"(55.40879821777344, 37.90629959106445, 588)","(55.606201171875, 49.278701782227, 411)"
4,Aerocondor,Belgorod International Airport,Kazan International Airport,Belgorod,Russia,Kazan,Russia,1008.253110,1.752488,474.126555,"(50.643798828125, 36.5900993347168, 735)","(55.606201171875, 49.278701782227, 411)"
...,...,...,...,...,...,...,...,...,...,...,...,...
36521,Iberworld,Lifou Airport,Tiga Airport,Lifou,New Caledonia,Tiga,New Caledonia,68.609280,0.585229,4.304640,"(-20.774799346923828, 167.24000549316406, 92)","(-21.096099853515625, 167.8040008544922, 128)"
36522,Iberworld,Nouméa Magenta Airport,Touho Airport,Noumea,New Caledonia,Touho,New Caledonia,205.972279,0.755866,72.986139,"(-22.25830078125, 166.47300720214844, 10)","(-20.790000915527344, 165.25900268554688, 10)"
36523,Iberworld,Nouméa Magenta Airport,Ouvéa Airport,Noumea,New Caledonia,Ouvea,New Caledonia,180.177492,0.723823,60.088746,"(-22.25830078125, 166.47300720214844, 10)","(-20.640600204467773, 166.572998046875, 23)"
36524,Iberworld,Lifou Airport,Ouvéa Airport,Lifou,New Caledonia,Ouvea,New Caledonia,70.962955,0.588153,5.481478,"(-20.774799346923828, 167.24000549316406, 92)","(-20.640600204467773, 166.572998046875, 23)"


In [67]:
def haversine(lat1, lon1, alt1, lat2, lon2, alt2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Earth radius in meters (you can use different values based on your dataset)
    R = 6371000

    # Altitude difference
    alt_diff = abs(alt1 - alt2)

    # Haversine formula
    a = math.sin((lat2 - lat1) / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin((lon2 - lon1) / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Calculate the distance
    distance = R * c + alt_diff

    # distance is in meteres, so we return the value in kilometers
    return distance / 1000 

In [68]:
class Node:
    def __init__(self, airport, city, country, coordination):
        self.airport = airport
        self.city = city
        self.country = country
        self.coordination = coordination


In [69]:
class Edge:
    def __init__(self, airline, distance, fly_time, price):
        self.airline = airline
        self.distance = distance
        self.fly_time = fly_time
        self.price = price

    def get_score(self, w1 = 1, w2 = 700, w3 = 2):
        return self.distance * w1 + self.fly_time * w2 + self.price * w3


In [70]:
dft = df[['Distance', 'FlyTime', 'Price']]

In [71]:
dft

,Distance,FlyTime,Price
0,1506.825604,2.371833,723.412802
1,1040.438320,1.792470,490.219160
2,770.508500,1.457153,355.254250
3,715.649350,1.389005,327.824675
4,1008.253110,1.752488,474.126555
...,...,...,...
36521,68.609280,0.585229,4.304640
36522,205.972279,0.755866,72.986139
36523,180.177492,0.723823,60.088746
36524,70.962955,0.588153,5.481478


In [72]:
dft.corr()

,Distance,FlyTime,Price
Distance,1.0,1.0,1.0
FlyTime,1.0,1.0,1.0
Price,1.0,1.0,1.0


In [73]:
nodes = {}
graph = {}

In [74]:
for data in df.values:
    airline, src_airport, dest_airport, src_city, src_country, dest_city, dest_country, distance, fly_time, price, src_loc, dest_loc = data
    if src_airport not in nodes:
        nodes[src_airport] = Node(src_airport, src_city, src_country, src_loc)
    
    if dest_airport not in nodes:
        nodes[dest_airport] = Node(dest_airport, dest_city, dest_country, dest_loc)
    
    e = Edge(airline, distance, fly_time, price)
    
    if src_airport not in graph:
        graph[src_airport] = {}
    
    if dest_airport not in graph:
        graph[dest_airport] = {}
        
    graph[src_airport][dest_airport] = e

In [75]:
def get_path_details(parent, start):
    path = []
    curr = start
    distance = 0
    fly_time = 0
    price = 0
    
    while start != -1:
        path.append(curr)

        if curr not in parent:
            return None # Return failure

        next = parent[curr]
        
        if next == -1:
            break
            
        e = graph[next][curr]
        
        distance += e.distance
        fly_time += e.fly_time
        price += e.price
        
        curr = next
        
    path = list(reversed(path))

    return path, distance, fly_time, price

In [90]:
def show_result(path, distance, fly_time, price, ex_time, algorithm):
    result = f"""{algorithm}
Exe time: {ex_time}s
.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
"""
    for i in range(len(path) - 1):
        edge = graph[path[i]][path[i + 1]]

        result += f"""Flight #{i + 1} ({edge.airline}):
From: {path[i]} - {nodes[path[i]].city}, {nodes[path[i]].country}
To: {path[i + 1]} - {nodes[path[i + 1]].city}, {nodes[path[i + 1]].country}
Duration: {edge.distance}km
Time: {edge.fly_time}km
Price: {edge.price}km
----------------------------
"""
    result += f"""Total Price: ${price}
Total Duration: {distance}km
Total Time: {fly_time}h
"""

    return result

In [77]:
def dijkstra(src, dest):
    s_time = time.time()
    pq = []
    heap.heappush(pq, (0, src))
    
    parent = {}
    dist = {}
    dist[src] = 0
    parent[src] = -1
    
    while pq:
            d, u = heap.heappop(pq)
            for v in graph[u]:
                weight = graph[u][v].get_score()
                if (v not in dist) or (dist[v] > dist[u] + weight):
                    dist[v] = dist[u] + weight
                    heap.heappush(pq, (dist[v], v))
                    parent[v] = u
    
    path_details = get_path_details(parent, dest)
    if path_details is None:
       return None

    path, distance, fly_time, price = path_details

    return (dist[dest], path, distance, fly_time, price, time.time() - s_time)

In [78]:
res = dijkstra('Imam Khomeini International Airport', 'Raleigh Durham International Airport')
show_result(res[1], res[2], res[3], res[4], res[5], "A*")

A*
Exe time: 0.04024004936218262s
.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
Flight #1 (Iran Air):
From: Imam Khomeini International Airport - Tehran, Iran
To: Amsterdam Airport Schiphol - Amsterdam, Netherlands
Duration: 4075.580681472172km
Time: 5.562833144685928km
Price: 2007.790340736086km
----------------------------
Flight #2 (easyJet):
From: Amsterdam Airport Schiphol - Amsterdam, Netherlands
To: Newcastle Airport - Newcastle, United Kingdom
Duration: 522.06486620626km
Time: 1.1485277841071553km
Price: 231.03243310313km
----------------------------
Flight #3 (Jetstar Airways):
From: Newcastle Airport - Newcastle, United Kingdom
To: Melbourne International Airport - Melbourne, Australia
Duration: 834.6449978671836km
Time: 1.5368260843070605km
Price: 387.3224989335918km
----------------------------
Flight #4 (American Airlines):
From: Melbourne International Airport - Melbourne, Australia
To: Charlotte Douglas International Airport - Charlotte, United States
Duration: 791.23

In [79]:
def heuristic(start, goal):
    node1 = nodes[start]
    node2 = nodes[goal]
    return haversine(node1.coordination[0], node1.coordination[1], node1.coordination[2], node2.coordination[0], node2.coordination[1], node2.coordination[2]) / 50

In [80]:
def a_star(src, dest):
    s_time = time.time()
    pq = []
    heap.heappush(pq, (0, 0, src))
    
    parent = {}
    parent[src] = -1

    dist = {}
    dist[src] = 0

    found = False

    count = 0

    while pq:
        s, d, u = heap.heappop(pq)

        # Goal Test
        if u == dest:
            found = True
            break

        for v in graph[u]:
                e = graph[u][v]
                h = heuristic(v, dest)
                score = d + e.get_score()
                if (v not in dist) or (v in dist and dist[v] >= score):
                    dist[v] = score
                    parent[v] = u
                    heap.heappush(pq, (score + h, score, v))

    if not found:
        return None # Return Failure

    path_details = get_path_details(parent, dest)
    if path_details is None:
       return None

    path, distance, fly_time, price = path_details

    return (dist[dest], path, distance, fly_time, price,  time.time() - s_time)
                

In [93]:
res = a_star('Imam Khomeini International Airport', 'Raleigh Durham International Airport')
if res is None:
    print("No path was found.")
else:
    result = show_result(res[1], res[2], res[3], res[4], res[5], "Dijkstra")
    print(result)

Dijkstra
Exe time: 0.09381747245788574s
.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
Flight #1 (Iran Air):
From: Imam Khomeini International Airport - Tehran, Iran
To: Amsterdam Airport Schiphol - Amsterdam, Netherlands
Duration: 4075.580681472172km
Time: 5.562833144685928km
Price: 2007.790340736086km
----------------------------
Flight #2 (easyJet):
From: Amsterdam Airport Schiphol - Amsterdam, Netherlands
To: Newcastle Airport - Newcastle, United Kingdom
Duration: 522.06486620626km
Time: 1.1485277841071553km
Price: 231.03243310313km
----------------------------
Flight #3 (Jetstar Airways):
From: Newcastle Airport - Newcastle, United Kingdom
To: Melbourne International Airport - Melbourne, Australia
Duration: 834.6449978671836km
Time: 1.5368260843070605km
Price: 387.3224989335918km
----------------------------
Flight #4 (American Airlines):
From: Melbourne International Airport - Melbourne, Australia
To: Charlotte Douglas International Airport - Charlotte, United States
Duration: 

In [94]:
res = dijkstra('Imam Khomeini International Airport', 'Raleigh Durham International Airport')
if res is None:
    print("No path was found.")
else:
    result = show_result(res[1], res[2], res[3], res[4], res[5], "Dijkstra")
    print(result)

Dijkstra
Exe time: 0.04931950569152832s
.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
Flight #1 (Iran Air):
From: Imam Khomeini International Airport - Tehran, Iran
To: Amsterdam Airport Schiphol - Amsterdam, Netherlands
Duration: 4075.580681472172km
Time: 5.562833144685928km
Price: 2007.790340736086km
----------------------------
Flight #2 (easyJet):
From: Amsterdam Airport Schiphol - Amsterdam, Netherlands
To: Newcastle Airport - Newcastle, United Kingdom
Duration: 522.06486620626km
Time: 1.1485277841071553km
Price: 231.03243310313km
----------------------------
Flight #3 (Jetstar Airways):
From: Newcastle Airport - Newcastle, United Kingdom
To: Melbourne International Airport - Melbourne, Australia
Duration: 834.6449978671836km
Time: 1.5368260843070605km
Price: 387.3224989335918km
----------------------------
Flight #4 (American Airlines):
From: Melbourne International Airport - Melbourne, Australia
To: Charlotte Douglas International Airport - Charlotte, United States
Duration: 